## Imports

In [1]:
# adds parent directory to python path so we can access code located there
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)
    
# core imports
from ohmeow_ml.keras_tf_util import *

# configure matplotlib
%matplotlib inline
    
# configure autoreload to re-load changed modules
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


## Define paths and global variables

In [2]:
current_dir = os.getcwd()
DATA_HOME_DIR = current_dir + '/data/'
DATA_CLASSES = [ dir for dir in os.listdir(DATA_HOME_DIR+'train') ]

# path = DATA_HOME_DIR
path = DATA_HOME_DIR + 'sample/'

train_path = path + 'train/'
val_path = path + 'valid/'
test_path = path + 'test/'

models_path = path + 'models/'                      # save weights here
results_path = path + 'results/'                    # save predictions here
processed_data_path = path + 'preprocesed_data/'    # save preprocessed data used for training here

if not os.path.exists(models_path): os.makedirs(models_path)
if not os.path.exists(results_path): os.makedirs(results_path)
if not os.path.exists(processed_data_path): os.makedirs(processed_data_path)

In [3]:
batch_size = 4 #64

## Preprocess the data

We can save time by pre-processing the images (e.g., converting them to jpegs, resizing to 224x224) and saving them as a numpy array on the file system.  We can do the same for the train, validation, and test image class designations, filenames, and one-hot encoded labels

In [4]:
# get classes, one-hot encoded labels, and filenames
train_classes, train_labels, train_filenames = get_batch_info(train_path)
val_classes, val_labels, val_filenames = get_batch_info(val_path)
test_filenames = get_batch_info(test_path)[2]

Found 1500 images belonging to 10 classes.
Found 750 images belonging to 10 classes.
Found 500 images belonging to 1 classes.


In [5]:
 # get image data
if not os.path.exists(processed_data_path+'train_data.bc'):
    train_data = get_data(train_path)
    save_array(processed_data_path+'train_data.bc', train_data)
else:
    train_data = load_array(processed_data_path+'train_data.bc')
    print('training data loaded ...')

if not os.path.exists(processed_data_path+'val_data.bc'):
    val_data = get_data(val_path)
    save_array(processed_data_path+'val_data.bc', val_data)
else:
    val_data = load_array(processed_data_path+'val_data.bc')
    print('validation data loaded ...')

if not os.path.exists(processed_data_path+'test_data.bc'):
    test_data = get_data(test_path)
    save_array(processed_data_path+'test_data.bc', test_data)
else:
    test_data = load_array(processed_data_path+'test_data.bc')
    print('test data loaded ...')

training data loaded ...
validation data loaded ...
test data loaded ...


Create training/validation batches and also define "steps per epoch" for each ... defines the # of batches per epoch (see `model.fit_generator()`).

***ONLY RUN THIS CODE IF YOU NEED TO USE BATCHES INSTEAD OF PERSISTED IMAGE ARRAYS***

In [6]:
# OPTION 1: BUILD BATCHES FROM FILE SYSTEM
# train_batches = get_batches(train_path, batch_size=batch_size)
# val_batches = get_batches(val_path, batch_size=batch_size*2, shuffle=False)

# OPTION 2: BUILD BATCHES FROM IMAGE ARRAYS
# gen = image.ImageDataGenerator()
# train_batches = gen.flow(train_data, train_labels, batch_size=batch_size, shuffle=True)
# val_batches = gen.flow(val_data, val_labels, batch_size=batch_size*2, shuffle=False)

# DEFINE # OF STEPS TO TAKE IN FITTING BATCHES FOR BOTH TRAINING AND VALIDATION EXAMPLES
# epoch_steps = math.ceil(train_batches.n/train_batches.batch_size)
# val_steps = math.ceil(val_batches.n/val_batches.batch_size)

## Basic Models
Train a linear classifer and a basic NN with a single hidden layer to provide a baseline and also validate that the size of our sample datasets are usable

### Option 1: A simple linear classifier.

In [7]:
def lm_model():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224,224,3)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
lm = lm_model()

In [9]:
# IF USING BATCHES ...
# lm.fit_generator(train_batches, steps_per_epoch=epoch_steps, epochs=2, 
#                  validation_data=val_batches, validation_steps=val_steps, verbose=2)

# IF USING IMAGE ARRAYS
lm.fit(train_data, train_labels, epochs=3, validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/3
6s - loss: 14.0125 - acc: 0.1087 - val_loss: 14.5580 - val_acc: 0.0920
Epoch 2/3
3s - loss: 14.0578 - acc: 0.1253 - val_loss: 14.5003 - val_acc: 0.0960
Epoch 3/3
3s - loss: 13.7063 - acc: 0.1453 - val_loss: 13.7745 - val_acc: 0.1413


In [11]:
# lm.summary()

While we have plenty of paramters (1,506,186 ~ 224\*224\*3\*10 = 1505280) our accuracy is really poor (~ .13)

NOTE: A **simple model with no regularization and plenty of parameters that doesn't perform well indicates that our learning rate is too high.**

From the notebook: "Perhaps it is jumping to a solution where it predicts one or two classes with high confidence so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate"

In [12]:
# IF USING BATCHES ...
# np.round(lm.predict_generator(train_batches, epoch_steps)[:10], 2)

# IF USING IMAGE ARRAYS
np.round(lm.predict(train_data, batch_size=batch_size)[:10],2)

array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

The above shows that indeed, using the standard learning rate of 0.001 is too high and causing the alorithm to select 1 most of the time.  If you see this, **lower the learning rate**

In [13]:
lm.optimizer.lr = 1e-05
lm.fit(train_data, train_labels, epochs=5, validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
3s - loss: 13.3597 - acc: 0.1700 - val_loss: 13.6901 - val_acc: 0.1467
Epoch 2/5
3s - loss: 13.7181 - acc: 0.1473 - val_loss: 14.6929 - val_acc: 0.0880
Epoch 3/5
3s - loss: 13.5963 - acc: 0.1547 - val_loss: 13.8928 - val_acc: 0.1347
Epoch 4/5
3s - loss: 13.4444 - acc: 0.1647 - val_loss: 14.3509 - val_acc: 0.1080
Epoch 5/5
3s - loss: 13.6139 - acc: 0.1547 - val_loss: 14.3430 - val_acc: 0.1053


BEST PRACTICE: **Start with a small learning rate, then increase really high, and then decrease it gradually by a factor of 10**

In [14]:
lm.optimizer.lr = 0.001
lm.fit(train_data, train_labels, epochs=5, validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
3s - loss: 13.5640 - acc: 0.1560 - val_loss: 14.5361 - val_acc: 0.0973
Epoch 2/5
3s - loss: 13.5402 - acc: 0.1573 - val_loss: 14.3184 - val_acc: 0.1107
Epoch 3/5
3s - loss: 13.5970 - acc: 0.1560 - val_loss: 14.1883 - val_acc: 0.1187
Epoch 4/5
3s - loss: 13.6925 - acc: 0.1500 - val_loss: 14.0030 - val_acc: 0.1307
Epoch 5/5
3s - loss: 13.5588 - acc: 0.1573 - val_loss: 14.3443 - val_acc: 0.1080


In [15]:
lm.optimizer.lr = 0.0001
lm.fit(train_data, train_labels, epochs=5, validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
3s - loss: 13.3760 - acc: 0.1687 - val_loss: 13.9368 - val_acc: 0.1333
Epoch 2/5
3s - loss: 13.4390 - acc: 0.1653 - val_loss: 13.9690 - val_acc: 0.1333
Epoch 3/5
3s - loss: 13.4918 - acc: 0.1607 - val_loss: 13.6667 - val_acc: 0.1520
Epoch 4/5
3s - loss: 13.4839 - acc: 0.1633 - val_loss: 13.9260 - val_acc: 0.1360
Epoch 5/5
3s - loss: 13.4344 - acc: 0.1653 - val_loss: 13.5684 - val_acc: 0.1573


### Option 2: A simple linear classifier with L2 regularization.

In [16]:
def build_lm_reg():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224,224,3)),
        Flatten(),
        Dense(10, activation='softmax', kernel_regularizer=l2(0.001))
    ])
    
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
lm = build_lm_reg()

In [18]:
lm.fit(train_data, train_labels, epochs=5, validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
3s - loss: 2.9119 - acc: 0.4393 - val_loss: 5.7881 - val_acc: 0.3800
Epoch 2/5
3s - loss: 0.5517 - acc: 0.8253 - val_loss: 4.6524 - val_acc: 0.3267
Epoch 3/5
3s - loss: 0.2400 - acc: 0.9327 - val_loss: 3.9197 - val_acc: 0.3720
Epoch 4/5
3s - loss: 0.0955 - acc: 0.9807 - val_loss: 3.1729 - val_acc: 0.4320
Epoch 5/5
3s - loss: 0.0582 - acc: 0.9893 - val_loss: 3.1366 - val_acc: 0.4387


## Validate Size of Sample

Once we are getting pretty consisten accuracy on our validation dataset, we should verify that our sample size is sufficient for further experiements.  If it isn't, adjust and run the previous code again

In [19]:
rnd_batches = get_batches(val_path, batch_size=batch_size*2, shuffle=True)
steps = math.ceil(rnd_batches.n / batch_size)
val_results = [ lm.evaluate_generator(rnd_batches, steps) for i in range(10) ]

Found 750 images belonging to 10 classes.


In [20]:
np.round(val_results, 2)

array([[ 3.14,  0.44],
       [ 3.16,  0.44],
       [ 3.14,  0.44],
       [ 3.12,  0.44],
       [ 3.16,  0.44],
       [ 3.12,  0.44],
       [ 3.14,  0.44],
       [ 3.13,  0.44],
       [ 3.11,  0.44],
       [ 3.15,  0.44]])

## NN Models for Sample

###  Single Hidden Layer

In [21]:
def nn():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224,224,3)),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    
    model.compile(Adam(lr=1e-05), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [22]:
model = nn()

In [23]:
model.fit(train_data, train_labels, batch_size=batch_size, epochs=5, 
          validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
14s - loss: 1.9553 - acc: 0.3553 - val_loss: 2.2258 - val_acc: 0.2587
Epoch 2/5
14s - loss: 1.2602 - acc: 0.6173 - val_loss: 2.0740 - val_acc: 0.3173
Epoch 3/5
14s - loss: 0.9435 - acc: 0.7620 - val_loss: 1.8366 - val_acc: 0.4120
Epoch 4/5
14s - loss: 0.7358 - acc: 0.8280 - val_loss: 2.0912 - val_acc: 0.3893
Epoch 5/5
14s - loss: 0.6254 - acc: 0.8653 - val_loss: 2.0172 - val_acc: 0.4200


In [25]:
model.optimizer.lr = 0.01
model.fit(train_data, train_labels, batch_size=batch_size, epochs=5, 
          validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
14s - loss: 0.5246 - acc: 0.8927 - val_loss: 2.0304 - val_acc: 0.3933
Epoch 2/5
14s - loss: 0.4643 - acc: 0.9133 - val_loss: 2.1392 - val_acc: 0.3947
Epoch 3/5
14s - loss: 0.4584 - acc: 0.9260 - val_loss: 1.7990 - val_acc: 0.4213
Epoch 4/5
14s - loss: 0.3634 - acc: 0.9480 - val_loss: 2.1439 - val_acc: 0.4240
Epoch 5/5
14s - loss: 0.3261 - acc: 0.9540 - val_loss: 2.0715 - val_acc: 0.3707


### Simple CNN

2 conv layers with max pooling + a simple dense network is a good simple CNN to start with

In [26]:
def simple_cnn():
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(224,224,3)),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3)),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])

    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [27]:
model = simple_cnn()

In [28]:
model.fit(train_data, train_labels, batch_size=batch_size, epochs=2, shuffle=True, 
          validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/2
53s - loss: 2.0548 - acc: 0.2933 - val_loss: 2.2419 - val_acc: 0.2307
Epoch 2/2
51s - loss: 1.2663 - acc: 0.6160 - val_loss: 2.1046 - val_acc: 0.3093


In [29]:
model.optimizer.lr = 0.001
model.fit(train_data, train_labels, batch_size=batch_size, epochs=5, shuffle=True, 
          validation_data=(val_data, val_labels), verbose=2)

Train on 1500 samples, validate on 750 samples
Epoch 1/5
51s - loss: 0.9307 - acc: 0.7533 - val_loss: 2.0036 - val_acc: 0.3587
Epoch 2/5
51s - loss: 0.7270 - acc: 0.8233 - val_loss: 1.9519 - val_acc: 0.4227
Epoch 3/5
51s - loss: 0.5729 - acc: 0.8740 - val_loss: 1.9649 - val_acc: 0.4000
Epoch 4/5
51s - loss: 0.4463 - acc: 0.9133 - val_loss: 1.9553 - val_acc: 0.4133
Epoch 5/5
52s - loss: 0.3936 - acc: 0.9293 - val_loss: 1.9389 - val_acc: 0.4333


## Data Augmentation

In [30]:
def test_augmentation(rotation_range=0.0, width_shift_range=0.0, height_shift_range=0.0, 
                      shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0,
                      horizontal_flip=False, vertical_flip=False):
    limit_mem()
    
    gen = image.ImageDataGenerator(
            rotation_range=rotation_range,           # degrees (0 to 180)
            width_shift_range=width_shift_range,     # fraction of total width
            height_shift_range=height_shift_range,   # fraction of total height
            shear_range=shear_range,                 # shear intensity (shear angle in radians; 2 radians = 360 degrees)
            zoom_range=zoom_range,                   # amount of zoom
            channel_shift_range=channel_shift_range, # shift range for each channels
            horizontal_flip=horizontal_flip, 
            vertical_flip=vertical_flip)
    
    da_batches = gen.flow(train_data, train_labels, batch_size=batch_size, shuffle=True)
    
    model = simple_cnn()
    
    epoch_steps = math.ceil(da_batches.n/da_batches.batch_size)
    model.fit_generator(da_batches, epoch_steps, epochs=2, validation_data=(val_data, val_labels), verbose=2)
    
    model.optimizer.lr = 0.001
    history = model.fit_generator(da_batches, epoch_steps, epochs=5, validation_data=(val_data, val_labels), verbose=2)
    
    return history

In [31]:
# define the types of data augmentations we want to test, and the values we want to test for each
aug_experiments = {
    'rotation_range' : [0, 2.5, 5, 10, 15],
    'width_shift_range': [0, 0.05, 1, 2, 4],
    'height_shift_range': [0, 0.05, 1, 2, 4],
    'shear_range': [0, 0.1, 0.15, 0.2, 0.3],
    'zoom_range': [0, 0.1, 0.15, 0.2, 0.3],
    'channel_shift_range': [0, 10, 20, 30, 50]
}

# used to store the results of data augmentation tests
df_augs = pd.DataFrame(columns=['aug', 'aug_val', 'train_loss', 'train_acc', 'val_loss', 'val_acc'])

# try each type of data augmentation one at a time
for k,v in aug_experiments.items():
    # for each type, try 3-4 different levels of augmentation
    for aug_val in v: 
        print('> {0} = {1}'.format(k, aug_val))
        h = test_augmentation(**{k:aug_val})

        # save the results of each tested value so that we can determine the best for
        # each data augmentation type
        df_augs = df_augs.append({
            'aug': k, 
            'aug_val': aug_val,
            'train_loss': np.mean(h.history['loss'][-3:]), 
            'train_acc': np.mean(h.history['acc'][-3:]), 
            'val_loss': np.mean(h.history['val_loss'][-3:]), 
            'val_acc': np.mean(h.history['val_acc'][-3:]) 
        }, ignore_index=True)

> rotation_range = 0
Epoch 1/2
52s - loss: 1.9792 - acc: 0.3393 - val_loss: 2.1059 - val_acc: 0.3053
Epoch 2/2
52s - loss: 1.1461 - acc: 0.6547 - val_loss: 2.0625 - val_acc: 0.3600
Epoch 1/5
51s - loss: 0.8226 - acc: 0.7793 - val_loss: 1.9166 - val_acc: 0.4413
Epoch 2/5
52s - loss: 0.6323 - acc: 0.8620 - val_loss: 1.9410 - val_acc: 0.4120
Epoch 3/5
52s - loss: 0.4968 - acc: 0.8920 - val_loss: 1.9140 - val_acc: 0.4160
Epoch 4/5
52s - loss: 0.4145 - acc: 0.9133 - val_loss: 1.9376 - val_acc: 0.4080
Epoch 5/5
52s - loss: 0.3509 - acc: 0.9373 - val_loss: 1.8453 - val_acc: 0.4520
> rotation_range = 1
Epoch 1/2
52s - loss: 2.0780 - acc: 0.3027 - val_loss: 2.2773 - val_acc: 0.2480
Epoch 2/2
52s - loss: 1.2817 - acc: 0.5967 - val_loss: 2.0705 - val_acc: 0.3213
Epoch 1/5
52s - loss: 0.9359 - acc: 0.7387 - val_loss: 1.9273 - val_acc: 0.4333
Epoch 2/5
52s - loss: 0.7145 - acc: 0.8113 - val_loss: 2.0870 - val_acc: 0.3933
Epoch 3/5
52s - loss: 0.5566 - acc: 0.8713 - val_loss: 1.9209 - val_acc: 0.457

Epoch 1/2
54s - loss: 2.5766 - acc: 0.1127 - val_loss: 2.7190 - val_acc: 0.1573
Epoch 2/2
52s - loss: 2.3819 - acc: 0.1333 - val_loss: 2.8952 - val_acc: 0.1107
Epoch 1/5
52s - loss: 2.3239 - acc: 0.1500 - val_loss: 2.7613 - val_acc: 0.1213
Epoch 2/5
52s - loss: 2.2748 - acc: 0.1953 - val_loss: 2.7353 - val_acc: 0.1107
Epoch 3/5
52s - loss: 2.2855 - acc: 0.1733 - val_loss: 2.9431 - val_acc: 0.1080
Epoch 4/5
52s - loss: 2.2583 - acc: 0.1853 - val_loss: 2.7123 - val_acc: 0.1267
Epoch 5/5
52s - loss: 2.2200 - acc: 0.1947 - val_loss: 3.0513 - val_acc: 0.1440
> shear_range = 0
Epoch 1/2
54s - loss: 2.0442 - acc: 0.3247 - val_loss: 2.0573 - val_acc: 0.3347
Epoch 2/2
52s - loss: 1.1625 - acc: 0.6580 - val_loss: 2.0630 - val_acc: 0.3920
Epoch 1/5
52s - loss: 0.8343 - acc: 0.7827 - val_loss: 1.8283 - val_acc: 0.4373
Epoch 2/5
52s - loss: 0.6243 - acc: 0.8620 - val_loss: 1.7455 - val_acc: 0.4533
Epoch 3/5
52s - loss: 0.5471 - acc: 0.8800 - val_loss: 1.8192 - val_acc: 0.4867
Epoch 4/5
52s - loss: 

52s - loss: 1.2685 - acc: 0.6140 - val_loss: 1.9876 - val_acc: 0.3720
Epoch 1/5
52s - loss: 0.9302 - acc: 0.7420 - val_loss: 1.8786 - val_acc: 0.4213
Epoch 2/5
52s - loss: 0.7264 - acc: 0.8167 - val_loss: 1.8552 - val_acc: 0.3907
Epoch 3/5
52s - loss: 0.5602 - acc: 0.8753 - val_loss: 1.7893 - val_acc: 0.4427
Epoch 4/5
52s - loss: 0.4805 - acc: 0.8933 - val_loss: 1.8501 - val_acc: 0.4400
Epoch 5/5
52s - loss: 0.4249 - acc: 0.9153 - val_loss: 1.9116 - val_acc: 0.4213
> channel_shift_range = 50
Epoch 1/2
56s - loss: 2.2244 - acc: 0.2553 - val_loss: 2.1832 - val_acc: 0.2413
Epoch 2/2
52s - loss: 1.5096 - acc: 0.5247 - val_loss: 2.0220 - val_acc: 0.3133
Epoch 1/5
52s - loss: 1.1497 - acc: 0.6547 - val_loss: 1.9722 - val_acc: 0.3600
Epoch 2/5
52s - loss: 0.9515 - acc: 0.7307 - val_loss: 1.8742 - val_acc: 0.3960
Epoch 3/5
52s - loss: 0.7264 - acc: 0.8080 - val_loss: 1.8761 - val_acc: 0.4040
Epoch 4/5
52s - loss: 0.6124 - acc: 0.8507 - val_loss: 1.8558 - val_acc: 0.4253
Epoch 5/5
52s - loss: 0

In [33]:
df_augs.to_csv(path+'data_augmentation_results.csv', index=False)

In [34]:
df_augs.sort_values('val_acc', ascending=False).groupby('aug').first()

,aug_val,train_loss,train_acc,val_loss,val_acc
aug,,,,,
channel_shift_range,10.00,0.464624,0.902444,1.735638,0.464444
height_shift_range,0.00,0.431795,0.917333,1.985304,0.436000
rotation_range,10.00,0.817478,0.770444,1.790981,0.476444
shear_range,0.00,0.431570,0.912667,1.761170,0.480444
width_shift_range,0.05,0.867643,0.755333,1.771413,0.515556
zoom_range,0.00,0.445235,0.899778,1.787687,0.477778


In [35]:
gen_aug = image.ImageDataGenerator(channel_shift_range=10.0, height_shift_range=0.0, rotation_range=10.0, 
                                   shear_range=0.10, width_shift_range=0.05, zoom_range=0.0)

aug_batches = gen_aug.flow(train_data, train_labels, batch_size=batch_size)
epoch_steps = math.ceil(aug_batches.n/aug_batches.batch_size)

In [36]:
limit_mem()
model = simple_cnn()
model.fit_generator(aug_batches, epoch_steps, epochs=2, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/2
56s - loss: 2.3160 - acc: 0.2233 - val_loss: 2.2394 - val_acc: 0.2280
Epoch 2/2
52s - loss: 1.8298 - acc: 0.3747 - val_loss: 2.2451 - val_acc: 0.2907


In [37]:
model.optimizer.lr = 0.001
model.fit_generator(aug_batches, epoch_steps, epochs=5, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/5
52s - loss: 1.6255 - acc: 0.4467 - val_loss: 2.1800 - val_acc: 0.3200
Epoch 2/5
52s - loss: 1.4682 - acc: 0.5220 - val_loss: 2.1050 - val_acc: 0.3680
Epoch 3/5
52s - loss: 1.3419 - acc: 0.5660 - val_loss: 2.0229 - val_acc: 0.3733
Epoch 4/5
52s - loss: 1.2859 - acc: 0.5867 - val_loss: 1.9969 - val_acc: 0.4347
Epoch 5/5
52s - loss: 1.1854 - acc: 0.6267 - val_loss: 1.9332 - val_acc: 0.4560


In [38]:
model.optimizer.lr = 0.0001
model.fit_generator(aug_batches, epoch_steps, epochs=5, validation_data=(val_data, val_labels), verbose=2)

Epoch 1/5
52s - loss: 1.1351 - acc: 0.6407 - val_loss: 1.7510 - val_acc: 0.4787
Epoch 2/5
52s - loss: 1.0652 - acc: 0.6787 - val_loss: 1.8536 - val_acc: 0.4787
Epoch 3/5
52s - loss: 1.0316 - acc: 0.6753 - val_loss: 1.7617 - val_acc: 0.4880
Epoch 4/5
52s - loss: 0.9348 - acc: 0.7100 - val_loss: 1.7955 - val_acc: 0.4480
Epoch 5/5
52s - loss: 0.8878 - acc: 0.7207 - val_loss: 1.6889 - val_acc: 0.5107
